In [1]:
import parcels
from parcels import Field,FieldSet,ParticleSet,Variable,JITParticle,AdvectionRK4,ErrorCode,BrownianMotion2D
import datetime
from datetime import timedelta as delta 
from glob import glob
import math as m
import xarray as xr
import numpy as np
from operator import attrgetter
import os

In [2]:
class PlasticParticle(JITParticle):  
    # Define a new particle class that contains three extra variables
    u = Variable('u', dtype = np.float32)
    v = Variable('v', dtype = np.float32)
    beached = Variable('beached', dtype =np.float32, initial = 0 )
    
def Speed(particle, fieldset, time):
    (u, v) = fieldset.UV[time,particle.depth, particle.lat, particle.lon]  

def Beached(particle, fieldset, time):
    if particle.u == 0. and particle.v == 0.:
        particle.beached = 1

def Diffusion(particle, fiedlset, time):
    if particle.beached == 0 :
        'Parcels implemented kernel : AdvectionRK4DiffusionM1':
        # RK4 terms
        (u1, v1) = fieldset.UV[time, particle.depth, particle.lat, particle.lon]
        lon1, lat1 = (particle.lon + u1 * .5 * particle.dt, particle.lat + v1 * .5 * particle.dt)
        (u2, v2) = fieldset.UV[time + .5 * particle.dt, particle.depth, lat1, lon1]
        lon2, lat2 = (particle.lon + u2 * .5 * particle.dt, particle.lat + v2 * .5 * particle.dt)
        (u3, v3) = fieldset.UV[time + .5 * particle.dt, particle.depth, lat2, lon2]
        lon3, lat3 = (particle.lon + u3 * particle.dt, particle.lat + v3 * particle.dt)
        (u4, v4) = fieldset.UV[time + particle.dt, particle.depth, lat3, lon3]

        # Wiener increment with zero mean and std of sqrt(dt)
        dWx = random.uniform(-1., 1.) * math.sqrt(math.fabs(particle.dt) * 3)
        dWy = random.uniform(-1., 1.) * math.sqrt(math.fabs(particle.dt) * 3)

        Kxp1 = fieldset.Kh_zonal[time, particle.depth, particle.lat, particle.lon + fieldset.dres]
        Kxm1 = fieldset.Kh_zonal[time, particle.depth, particle.lat, particle.lon - fieldset.dres]
        dKdx = (Kxp1 - Kxm1) / (2 * fieldset.dres)
        bx = math.sqrt(2 * fieldset.Kh_zonal[time, particle.depth, particle.lat, particle.lon])

        Kyp1 = fieldset.Kh_meridional[time, particle.depth, particle.lat + fieldset.dres, particle.lon]
        Kym1 = fieldset.Kh_meridional[time, particle.depth, particle.lat - fieldset.dres, particle.lon]
        dKdy = (Kyp1 - Kym1) / (2 * fieldset.dres)
        by = math.sqrt(2 * fieldset.Kh_meridional[time, particle.depth, particle.lat, particle.lon])

        # Particle positions are updated only after evaluating all terms.
        particle.lon += ((u1 + 2 * u2 + 2 * u3 + u4) / 6.) * particle.dt + 0.5 * dKdx * (dWx**2 + particle.dt) + bx * dWx
        particle.lat += ((v1 + 2 * v2 + 2 * v3 + v4) / 6.) * particle.dt + 0.5 * dKdy * (dWy**2 + particle.dt) + by * dWy


def periodicBC(particle, fieldset, time):
    """
    Kernel for periodic values in longitude
    """
    if particle.lon < 0.:
        particle.lon += 360.
    elif particle.lon >= 360.:
        particle.lon -= 360.
        
def OutOfBounds(particle, fieldset, time):
    particle.delete()
    
def set_currents (uvfiles):
    #all in the same file
    filenames = {'U': uvfiles,
            'V': uvfiles}
    dimensions = {'lon': 'longitude', 'lat': 'latitude', 'time': 'time'}
    variables = {'U':'uo', 'V' : 'vo'}
    uvindices = {'lon': range(2484, 2664), 'lat': range(1452, 1536)} #define domain 
   
    
    fset_currents = FieldSet.from_netcdf(filenames, variables, dimensions,allow_time_extrapolation = False, 
                                    indices = uvindices)
    fset_currents.add_periodic_halo(zonal=True, meridional=False, halosize=5)
    
    return fset_currents


def set_stokes(stokesfiles):
    stokesfilenames = {'U': stokesfiles,
            'V': stokesfiles}
    stokesdimensions = {'lon': 'longitude','lat': 'latitude', 'time': 'time'}
    stokesvariables ={'U': 'uuss', 'V': 'uuss'}
    
    stokesindices = {'lon': range(420, 440), 'lat': range(237, 253)} #define domain    
    

    fset_stokes = FieldSet.from_netcdf(stokesfiles,stokesvariables,stokesdimensions,
                                       allow_time_extrapolation = False, indices = stokesindices)
    fset_stokes.add_periodic_halo(zonal=True, meridional=False, halosize=5)

    
    return fset_stokes


In [3]:
## Load data

outdir= '/science-nfs-sys/vsm01/users/6312454/' + 'BS_grid'
datadir = '/data/oceanparcels/input_data'

stokesfiles = sorted(glob(datadir+'/WaveWatch3data/CFSR/WW3-GLOB-30M_*.nc'))
uvfiles = sorted(glob(datadir +'/CMEMS/GLOBAL_REANALYSIS_PHY_001_030/mercatorglorys12v1_gl12_mean_*.nc'))


In [4]:
## Fiedlset 
currents = set_currents(uvfiles)
stokes = set_stokes(stokesfiles)

fset = FieldSet(U = currents.U, V = currents.V)
fset_stokes = FieldSet(U =currents.U + stokes.U, V = currents.V + stokes.V)

In [5]:
#Add field for BM2D

size2D = (fset.U.grid.ydim, fset.U.grid.xdim)
KH_z = 0.25
alpha = 1
L = fset.U.grid.ydim
beta = np.zeros(fset.U.grid.ydim)
y_K = fset.V.grid.lat

for yi in range(len(y_K)):
    if y_K[yi] < L/2:
        beta[yi] = y_K[yi]*np.power(L - 2*y_K[yi], 1/alpha)
    elif y_K[yi] >= L/2:
        beta[yi] = (L - y_K[yi])*np.power(2*y_K[yi] - L, 1/alpha)

KH_m = KH_z*(2*(1+alpha)*(1+2*alpha))/(alpha**2*np.power(L, 1+1/alpha))*beta
Kh_meridional = np.array([KH_m for x in range(190)]).T


fset.add_field(Field('Kh_zonal', data=KH_z*np.ones(size2D),lat=fset.U.grid.lat,lon=fset.U.grid.lon,
                     mesh='spherical', allow_time_extrapolation=True))
fset.add_field(Field('Kh_meridional', data=Kh_meridional*np.ones(size2D), 
                     lon=fset.U.grid.lon,lat=fset.U.grid.lat,mesh='spherical', allow_time_extrapolation=True))

fset.add_constant('dres', 0.00002)

fset_stokes.add_field(Field('Kh_zonal', data=KH_z*np.ones(size2D),lat=fset.U.grid.lat,lon=fset.U.grid.lon,
                     mesh='spherical', allow_time_extrapolation=True))
fset_stokes.add_field(Field('Kh_meridional', data=Kh_meridional*np.ones(size2D), 
                            lon=fset.U.grid.lon,lat=fset.U.grid.lat,mesh='spherical', 
                            allow_time_extrapolation=True))

fset_stokes.add_constant('dres', 0.00002)

In [6]:
### RIVERS
Danube1    = [45.221998, 29.737916]
Danube2    = [45.172493, 29.727552] #Sulina branch 
Danube3    = [44.877123, 29.625403] #Saint George branch
Dniepr     = [46.555790, 32.026125]
Rioni1     = [42.211490, 41.630494]
Rioni2     = [42.183748, 41.634994]
Dniester   = [46.081485, 30.494232]
Coruh      = [41.608372, 41.560132]
Kizilirmak = [41.754870, 35.955972]
Sakarya    = [41.136737, 30.650267]
Yesilimark = [41.386579, 36.661469]
Kodori     = [42.819377, 41.129852]
Bzyb       = [43.184802, 40.277446]


lats = np.array([Danube1[0],Danube2[0],Danube3[0], Dniepr[0], Rioni1[0], Rioni2[0], Dniester[0],
                     Coruh[0], Kizilirmak[0], Sakarya[0], Yesilimark[0], Kodori[0], Bzyb[0]])
lons = np.array([Danube1[1],Danube2[1],Danube3[1], Dniepr[1], Rioni1[1], Rioni2[1], Dniester[1],
                     Coruh[1], Kizilirmak[1], Sakarya[1], Yesilimark[1], Kodori[1], Bzyb[1]])



In [31]:
###ADVECTION

#startlats = lats
#startlons = lons



startlats = np.hstack([np.repeat(lats[0:3],[33/3]), np.repeat(lats[3],[7.5]), np.repeat(lats[4:6],[2/2]),
                     np.repeat(lats[6],[2]), np.repeat(lats[7],[1.5]), np.repeat(lats[8],[1]),
                     np.repeat(lats[8],[1]), np.repeat(lats[9],[2]), np.repeat(lats[10],[1]), 
                     np.repeat(lats[12],[1])])

startlons = np.hstack([np.repeat(lons[0:3],[33/3]), np.repeat(lons[3],[7.5]), np.repeat(lons[4:6],[2/2]),
                     np.repeat(lons[6],[2]), np.repeat(lons[7],[1.5]), np.repeat(lons[8],[1]),
                     np.repeat(lons[8],[1]), np.repeat(lats[9],[2]), np.repeat(lons[10],[1]), 
                     np.repeat(lons[12],[1])])

startlons = [l + 360 for l in startlons]


rundays = 365
dt = 30        #[m]
#nperloc = 7.5

starttime = datetime.datetime(2015,1,1,0,0)
starttime = np.repeat(starttime, len(startlats))

repeatdt = delta(days=1)


psetname ='RUN_RIVERS4_weighted_'

name = str(psetname)+str(rundays)+'d_'+str(dt)+'h'   #outputfile name


pset_RIVERS = ParticleSet(fieldset=fset_stokes, pclass=PlasticParticle, lon=startlons,
                          #np.repeat(startlons, [nperloc]),
                          lat=startlats,
                          #np.repeat(startlats, [nperloc]), 
                          time = starttime,
                          #time=np.repeat(starttime, [nperloc]), 
                          repeatdt=repeatdt)

kernels = pset_RIVERS.Kernel(AdvectionRK4) + periodicBC + Speed + Beached + Diffusion

In [ ]:
pset_RIVERS.execute(kernels, runtime = delta(days = rundays), dt = delta(minutes= dt),
                      output_file = pset_RIVERS.ParticleFile(name = name,
                      outputdt=delta(hours=12)), 
                      recovery = {ErrorCode.ErrorOutOfBounds: OutOfBounds}, 
                      verbose_progress =False)


INFO: Compiled PlasticParticleAdvectionRK4periodicBCSpeedBeachedDiffusion ==> /tmp/parcels-266161/23bb759b1322c11df5687969d000df0e_0.so


In [7]:
#Danube/Dniepr

Danube1 = [45.221998, 29.737916]
Danube2 = [45.172493, 29.727552] #Sulina branch 
Danube3 = [44.877123, 29.625403] #Saint George branch
Dniepr  = [46.555790, 32.026125]


lats = np.array([Danube1[0],Danube2[0],Danube3[0],Dniepr[0]])
lons = np.array([Danube1[1],Danube2[1],Danube3[1],Dniepr[1]])

startlats = np.hstack([np.repeat(lats[0:2], 66/3), np.repeat(lats[2],15)])
startlons = np.hstack([np.repeat(lons[0:2], 66/3), np.repeat(lons[2],15)])
startlons = [l + 360 for l in startlons]


rundays = 365
dt = 30         #[m]
#nperloc = 66/3

starttime = datetime.datetime(2015,1,1,0,0)
starttime = np.repeat(starttime, len(startlats))

repeatdt = delta(days=1)



psetname ='RUN_DANUBE_DNIEPR_'

name = str(psetname)+str(rundays)+'d_'+str(dt)+'h'   #outputfile name


pset_DANUBE = ParticleSet(fieldset=fset_stokes, pclass=PlasticParticle, 
                          lon=startlons,lat=startlats,time = starttime,repeatdt=repeatdt)

kernels = pset_DANUBE.Kernel(AdvectionRK4) + periodicBC + Speed + Beached + Diffusion

pset_DANUBE.execute(kernels, runtime = delta(days = rundays), dt = delta(minutes= dt),
                    output_file = pset_DANUBE.ParticleFile(name = name, outputdt=delta(hours=12)), 
                    recovery = {ErrorCode.ErrorOutOfBounds: OutOfBounds},verbose_progress =False)

INFO: Compiled PlasticParticleAdvectionRK4periodicBCSpeedBeachedDiffusion ==> /tmp/parcels-266161/02841bc508639a0257c07ea7ef777f8c_0.so


In [9]:
#Danube

Danube1 = [45.221998, 29.737916]
Danube2 = [45.172493, 29.727552] #Sulina branch 
Danube3 = [44.877123, 29.625403] #Saint George branch
 
lats = np.array([Danube1[0],Danube2[0],Danube3[0]])
lons = np.array([Danube1[1],Danube2[1],Danube3[1]])

startlats = lats
startlons = lons
startlons = [l + 360 for l in startlons]


rundays = 365
dt = 30         #[m]
nperloc = 66/3

starttime = datetime.datetime(2015,1,1,0,0)
starttime = np.repeat(starttime, len(startlats))

repeatdt = delta(days=1)



psetname ='RUN_DANUBE4_'

name = str(psetname)+str(rundays)+'d_'+str(dt)+'h'   #outputfile name


pset_DANUBE = ParticleSet(fieldset=fset_stokes, pclass=PlasticParticle, lon=np.repeat(startlons, [nperloc]),
                   lat=np.repeat(startlats, [nperloc]), time=np.repeat(starttime, [nperloc]), 
                          repeatdt = repeatdt)

kernels = pset_DANUBE.Kernel(AdvectionRK4) + periodicBC + Speed + Beached + Diffusion

pset_DANUBE.execute(kernels, runtime = delta(days = rundays), dt = delta(minutes= dt),
                    output_file = pset_DANUBE.ParticleFile(name = name, outputdt=delta(hours=12)), 
                    recovery = {ErrorCode.ErrorOutOfBounds: OutOfBounds},verbose_progress =False)


INFO: Compiled PlasticParticleAdvectionRK4periodicBCSpeedBeachedDiffusion ==> /tmp/parcels-266161/1408b296e5ce59b359203b353f090960_0.so


In [11]:
#Dniepr

Dniepr = [46.511751, 32.271580]

lats = np.array([Dniepr[0]])
lons = np.array([Dniepr[1]])

startlats = lats
startlons = lons
startlons = [l + 360 for l in startlons]

y = 2012
m = 1
d = 1
rundays = 365
dt = 30         #[m]
nperloc = 15


starttime = datetime.datetime(2015,1,1,0,0)
starttime = np.repeat(starttime, len(startlats))


#t = [datetime(y, m, d)]*len(startlons)

repeatdt = delta(days=1)


psetname ='RUN_DNIEPR4_'

name = str(psetname)+str(rundays)+'d_'+str(dt)+'h'   #outputfile name


pset_DNIEPR = ParticleSet(fieldset=fset_stokes, pclass=PlasticParticle, lon=np.repeat(startlons, [nperloc]),
                   lat=np.repeat(startlats, [nperloc]), time=np.repeat(starttime, [nperloc]),
                    repeatdt=repeatdt)

kernels = pset_DNIEPR.Kernel(AdvectionRK4) + periodicBC + Speed + Beached + Diffusion

pset_DNIEPR.execute(kernels, runtime = delta(days = rundays), dt = delta(minutes= dt),
                      output_file = pset_DNIEPR.ParticleFile(name = name,
                      outputdt=delta(hours=12)), 
                      recovery = {ErrorCode.ErrorOutOfBounds: OutOfBounds}, 
                      verbose_progress =False)

INFO: Compiled PlasticParticleAdvectionRK4periodicBCSpeedBeachedDiffusion ==> /tmp/parcels-266161/281a9013763eb82c52e074d833d5256b_0.so


In [7]:
Rioni1 = [42.211490, 41.630494]
Rioni2 = [42.183748, 41.634994]

lats = np.array([Rioni1[0],Rioni2[0]])
lons = np.array([Rioni1[1],Rioni2[1]])

startlats = lats
startlons = lons
startlons = [l + 360 for l in startlons]

y = 2012
m = 1
d = 1
rundays = 365
dt = 30         #[m]
nperloc = 70/2

starttime = datetime.datetime(2015,1,1,0,0)
starttime = np.repeat(starttime, len(startlats))


#t = [datetime(y, m, d)]*len(startlons)

repeatdt = delta(days=1)
#time = np.arange(0, npart) * delta(hours=1).total_seconds()  # release every particle one hour later


psetname ='RUN_RIONI5_'

name = str(psetname)+str(rundays)+'d_'+str(dt)+'h'   #outputfile name


pset_RIONI = ParticleSet(fieldset=fset_stokes, pclass=PlasticParticle, lon=np.repeat(startlons, [nperloc]),
                   lat=np.repeat(startlats, [nperloc]), time=np.repeat(starttime, [nperloc]),
                         repeatdt=repeatdt)

kernels = pset_RIONI.Kernel(AdvectionRK4) + periodicBC + Speed + Beached + Diffusion

pset_RIONI.execute(kernels, runtime = delta(days = rundays), dt = delta(minutes= dt),
                      output_file = pset_RIONI.ParticleFile(name = name,
                      outputdt=delta(hours=12)), 
                      recovery = {ErrorCode.ErrorOutOfBounds: OutOfBounds}, 
                      verbose_progress =False)

INFO: Compiled PlasticParticleAdvectionRK4periodicBCSpeedBeachedDiffusion ==> /tmp/parcels-266161/c39135acabd669a136a09310f9a88b68_0.so


In [ ]:
Dniester = [46.081485, 30.494232]

lats = np.array([Dniester[0]])
lons = np.array([ Dniester[1]])

startlats = lats
startlons = lons
startlons = [l + 360 for l in startlons]

y = 2012
m = 1
d = 1
rundays = 365
dt = 30         #[m]
nperloc = 58

starttime = datetime.datetime(2015,1,1,0,0)
starttime = np.repeat(starttime, len(startlats))


repeatdt = delta(days=1)
#time = np.arange(0, npart) * delta(hours=1).total_seconds()  # release every particle one hour later


psetname ='RUN_DNIESTER5_'

name = str(psetname)+str(rundays)+'d_'+str(dt)+'h'   #outputfile name


pset_DNIESTER = ParticleSet(fieldset=fset_stokes, pclass=PlasticParticle, lon=np.repeat(startlons, [nperloc]),
                   lat=np.repeat(startlats, [nperloc]), time=np.repeat(starttime, [nperloc]),
                         repeatdt=repeatdt)

kernels = pset_DNIESTER.Kernel(AdvectionRK4) + periodicBC + Speed + Beached + Diffusion

pset_DNIESTER.execute(kernels, runtime = delta(days = rundays), dt = delta(minutes= dt),
                      output_file = pset_DNIESTER.ParticleFile(name = name,
                      outputdt=delta(hours=12)), 
                      recovery = {ErrorCode.ErrorOutOfBounds: OutOfBounds}, 
                      verbose_progress =False)

INFO: Compiled PlasticParticleAdvectionRK4periodicBCSpeedBeachedDiffusion ==> /tmp/parcels-266161/e93058aa569d1c33e22cb78ddc3d2d43_0.so


In [ ]:
Coruh = [41.608372, 41.560132]

lats = np.array([Coruh[0]])
lons = np.array([Coruh[1]])

startlats = lats
startlons = lons
startlons = [l + 360 for l in startlons]


rundays = 365
dt = 30         #[h]
nperloc = 47

starttime = datetime.datetime(2015,1,1,0,0)
starttime = np.repeat(starttime, len(startlats))



repeatdt = delta(days=1)
#time = np.arange(0, npart) * delta(hours=1).total_seconds()  # release every particle one hour later


psetname ='RUN_CORUH5_'

name = str(psetname)+str(rundays)+'d_'+str(dt)+'h'   #outputfile name


pset_CORUH = ParticleSet(fieldset=fset_stokes, pclass=PlasticParticle, lon=np.repeat(startlons, [nperloc]),
                   lat=np.repeat(startlats, [nperloc]), time=np.repeat(starttime, [nperloc]),
                         repeatdt=repeatdt)

kernels = pset_CORUH.Kernel(AdvectionRK4) + periodicBC + Speed + Beached + Diffusion

pset_CORUH.execute(kernels, runtime = delta(days = rundays), dt = delta(minutes= dt),
                      output_file = pset_CORUH.ParticleFile(name = name,
                      outputdt=delta(hours=12)), 
                      recovery = {ErrorCode.ErrorOutOfBounds: OutOfBounds}, 
                      verbose_progress =False)

In [ ]:
Kizilirmak = [41.754870, 35.955972]

lats = np.array([Kizilirmak[0]])
lons = np.array([Kizilirmak[1]])

startlats = lats
startlons = lons
startlons = [l + 360 for l in startlons]

y = 2012
m = 1
d = 1
rundays = 365
dt = 30        #[h]
nperloc = 30


starttime = datetime.datetime(2015,1,1,0,0)
starttime = np.repeat(starttime, len(startlats))


repeatdt = delta(days=1)
#time = np.arange(0, npart) * delta(hours=1).total_seconds()  # release every particle one hour later


psetname ='RUN_KIZILIRMAK5_'

name = str(psetname)+str(rundays)+'d_'+str(dt)+'h'   #outputfile name


pset_KIZILIRMAK = ParticleSet(fieldset=fset_stokes, pclass=PlasticParticle, lon=np.repeat(startlons, [nperloc]),
                   lat=np.repeat(startlats, [nperloc]), time=np.repeat(starttime, [nperloc]),repeatdt=repeatdt)

kernels = pset_KIZILIRMAK.Kernel(AdvectionRK4) + periodicBC + Speed + Beached + Diffusion

pset_KIZILIRMAK.execute(kernels, runtime = delta(days = rundays), dt = delta(minutes= dt),
                      output_file = pset_KIZILIRMAK.ParticleFile(name = name,
                      outputdt=delta(hours=12)), 
                      recovery = {ErrorCode.ErrorOutOfBounds: OutOfBounds}, 
                      verbose_progress =False)


In [ ]:
Sakarya = [41.136737, 30.650267]

lats = np.array([Sakarya[0]])
lons = np.array([Sakarya[1]])

startlats = lats
startlons = lons
startlons = [l + 360 for l in startlons]

y = 2012
m = 1
d = 1
rundays = 365
dt = 2         #[h]
nperloc = 32

starttime = datetime.datetime(2015,1,1,0,0)
starttime = np.repeat(starttime, len(startlats))


repeatdt = delta(days=1)
#time = np.arange(0, npart) * delta(hours=1).total_seconds()  # release every particle one hour later


psetname ='RUN_SAKARYA5_'

name = str(psetname)+str(rundays)+'d_'+str(dt)+'h'   #outputfile name


pset_SAKARYA = ParticleSet(fieldset=fset_stokes, pclass=DistParticle, lon=np.repeat(startlons, [nperloc]),
                   lat=np.repeat(startlats, [nperloc]), time=np.repeat(starttime, [nperloc]), repeatdt=repeatdt)

kernels = pset_SAKARYA.Kernel(AdvectionRK4) + periodicBC + TotalDistance + BrownianMotion2D

pset_SAKARYA.execute(kernels, runtime = delta(days = rundays), dt = delta(hours= dt),
                      output_file = pset_SAKARYA.ParticleFile(name = name,
                      outputdt=delta(hours=12)), 
                      recovery = {ErrorCode.ErrorOutOfBounds: OutOfBounds}, 
                      verbose_progress =False)



In [ ]:
Yesilimark = [41.386579, 36.661469]

lats = np.array([Yesilimark[0]])
lons = np.array([Yesilimark[1]])

startlats = lats
startlons = lons
startlons = [l + 360 for l in startlons]

y = 2012
m = 1
d = 1
rundays = 365
dt = 2         #[h]
nperloc = 29

starttime = datetime.datetime(2015,1,1,0,0)
starttime = np.repeat(starttime, len(startlats))


#t = [datetime(y, m, d)]*len(startlons)

repeatdt = delta(days=1)
#time = np.arange(0, npart) * delta(hours=1).total_seconds()  # release every particle one hour later


psetname ='RUN_YESILIMARK5_'

name = str(psetname)+str(rundays)+'d_'+str(dt)+'h'   #outputfile name


pset_YESILIMARK = ParticleSet(fieldset=fset_stokes, pclass=DistParticle, lon=np.repeat(startlons, [nperloc]),
                   lat=np.repeat(startlats, [nperloc]), time=np.repeat(starttime, [nperloc]), repeatdt=repeatdt)

kernels = pset_YESILIMARK.Kernel(AdvectionRK4) + periodicBC + TotalDistance + BrownianMotion2D

pset_YESILIMARK.execute(kernels, runtime = delta(days = rundays), dt = delta(hours= dt),
                      output_file = pset_YESILIMARK.ParticleFile(name = name,
                      outputdt=delta(hours=12)), 
                      recovery = {ErrorCode.ErrorOutOfBounds: OutOfBounds}, 
                      verbose_progress =False)


In [ ]:
Kodori = [42.819377, 41.129852]

lats = np.array([Kodori[0]])
lons = np.array([Kodori[1]])

startlats = lats
startlons = lons
startlons = [l + 360 for l in startlons]

y = 2012
m = 1
d = 1
rundays = 365
dt = 2         #[h]
nperloc = 23

starttime = datetime.datetime(2015,1,1,0,0)
starttime = np.repeat(starttime, len(startlats))

repeatdt = delta(days=1)
#time = np.arange(0, npart) * delta(hours=1).total_seconds()  # release every particle one hour later


psetname ='RUN_KODORI5_'

name = str(psetname)+str(rundays)+'d_'+str(dt)+'h'   #outputfile name


pset_KODORI = ParticleSet(fieldset=fset_stokes, pclass=DistParticle, lon=np.repeat(startlons, [nperloc]),
                   lat=np.repeat(startlats, [nperloc]), time=np.repeat(starttime, [nperloc]), repeatdt=repeatdt)

kernels = pset_KODORI.Kernel(AdvectionRK4) + periodicBC + TotalDistance + BrownianMotion2D

pset_KODORI.execute(kernels, runtime = delta(days = rundays), dt = delta(hours= dt),
                      output_file = pset_KODORI.ParticleFile(name = name,
                      outputdt=delta(hours=12)), 
                      recovery = {ErrorCode.ErrorOutOfBounds: OutOfBounds}, 
                      verbose_progress =False)



In [ ]:
Bzyb = [43.184802, 40.277446]

lats = np.array([Bzyb[0]])
lons = np.array([Bzyb[1]])

startlats = lats
startlons = lons
startlons = [l + 360 for l in startlons]

y = 2012
m = 1
d = 1
rundays = 365
dt = 30         #[h]
nperloc = 20

starttime = datetime.datetime(2015,1,1,0,0)
starttime = np.repeat(starttime, len(startlats))


repeatdt = delta(days=1)
#time = np.arange(0, npart) * delta(hours=1).total_seconds()  # release every particle one hour later


psetname ='RUN_BZYB5_'

name = str(psetname)+str(rundays)+'d_'+str(dt)+'h'   #outputfile name


pset_BZYB = ParticleSet(fieldset=fset_stokes, pclass=PlasticParticle, lon=np.repeat(startlons, [nperloc]),
                   lat=np.repeat(startlats, [nperloc]), time=np.repeat(starttime, [nperloc]), repeatdt=repeatdt)

kernels = pset_BZYB.Kernel(AdvectionRK4) + periodicBC + Speed + Beached + Diffusion

pset_BZYB.execute(kernels, runtime = delta(days = rundays), dt = delta(minutes= dt),
                      output_file = pset_BZYB.ParticleFile(name = name,
                      outputdt=delta(hours=12)), 
                      recovery = {ErrorCode.ErrorOutOfBounds: OutOfBounds}, 
                      verbose_progress =False)
